In [ ]:

"""
Data Transformation & Storage Script for Telco Churn
"""

import pandas as pd
import sqlite3
from pathlib import Path
from sklearn.preprocessing import MinMaxScaler

PROJECT_ROOT = Path.cwd().resolve().parent   # telco_churn_project
CLEAN_FILE = PROJECT_ROOT / "5.DataPreparation" / "clean" / "clean_telco.csv"
FEATURES_DIR = PROJECT_ROOT / "6.Transformation" / "features"
DB_FILE = FEATURES_DIR / "telco_features.db"
OUTPUT_FILE = FEATURES_DIR / "features_telco.csv"

FEATURES_DIR.mkdir(parents=True, exist_ok=True)

# -----------------------------
# Feature Engineering
# -----------------------------
def transform_data(input_file: Path, output_file: Path, db_file: Path):
    if not input_file.exists():
        raise FileNotFoundError(f" Clean file not found: {input_file}")

    df = pd.read_csv(input_file)

    # 1. Create aggregated features
    df["TotalSpendApprox"] = df["MonthlyCharges"] * df["tenure"]

    # 2. Derive new features
    df["AvgMonthlySpend"] = df["TotalCharges"] / (df["tenure"] + 1)  # avoid div/0
    df["IsSeniorCitizen"] = (df["SeniorCitizen"] == 1).astype(int)

    # 3. Normalize selected numeric features
    num_cols = ["tenure", "MonthlyCharges", "TotalCharges", "TotalSpendApprox", "AvgMonthlySpend"]
    scaler = MinMaxScaler()
    df[num_cols] = scaler.fit_transform(df[num_cols])

    # Save to CSV
    df.to_csv(output_file, index=False)
    print(f"Features saved at {output_file}")

    # Save to SQLite DB
    conn = sqlite3.connect(db_file)
    df.to_sql("telco_features", conn, if_exists="replace", index=False)
    conn.commit()
    conn.close()
    print(f"Features stored in SQLite database at {db_file}")

    return df


# -----------------------------
# Run
# -----------------------------
transformed_df = transform_data(CLEAN_FILE, OUTPUT_FILE, DB_FILE)
transformed_df.head()


Features saved at C:\Users\Gogula Vinay\telco_churn_project\06_transformation\features\features_telco.csv
Features stored in SQLite database at C:\Users\Gogula Vinay\telco_churn_project\06_transformation\features\telco_features.db


,customerID,SeniorCitizen,tenure,MonthlyCharges,TotalCharges,Churn,gender_Male,Partner_Yes,Dependents_Yes,PhoneService_Yes,...,StreamingMovies_Yes,Contract_One year,Contract_Two year,PaperlessBilling_Yes,PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check,TotalSpendApprox,AvgMonthlySpend,IsSeniorCitizen
0,7590-VHVEG,0,0.000000,0.115423,0.001275,0,False,True,False,False,...,False,False,False,True,False,True,False,0.736492,0.893080,0
1,5575-GNVDE,0,0.464789,0.385075,0.215867,0,True,False,False,True,...,False,True,False,False,False,False,True,0.454351,0.876131,0
2,3668-QPYBK,0,0.014085,0.354229,0.010310,1,True,False,False,True,...,False,False,False,True,False,False,True,0.542117,0.895177,0
3,7795-CFOCW,0,0.619718,0.239303,0.210241,0,True,False,False,False,...,False,True,False,False,False,False,False,0.385602,0.876287,0
4,9237-HQITU,0,0.014085,0.521891,0.015330,1,False,False,False,True,...,False,False,False,True,False,True,False,0.411883,0.894811,0


In [3]:
import sqlite3
import pandas as pd

# Connect to the SQLite DB
conn = sqlite3.connect(DB_FILE)

# List all tables
tables = pd.read_sql("SELECT name FROM sqlite_master WHERE type='table';", conn)
print("📂 Tables in DB:")
print(tables)

# Preview first 5 rows from telco_features
sample = pd.read_sql("SELECT * FROM telco_features LIMIT 5;", conn)
conn.close()

sample


📂 Tables in DB:
             name
0  telco_features


,customerID,SeniorCitizen,tenure,MonthlyCharges,TotalCharges,Churn,gender_Male,Partner_Yes,Dependents_Yes,PhoneService_Yes,...,StreamingMovies_Yes,Contract_One year,Contract_Two year,PaperlessBilling_Yes,PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check,TotalSpendApprox,AvgMonthlySpend,IsSeniorCitizen
0,7590-VHVEG,0,0.000000,0.115423,0.001275,0,0,1,0,0,...,0,0,0,1,0,1,0,0.736492,0.893080,0
1,5575-GNVDE,0,0.464789,0.385075,0.215867,0,1,0,0,1,...,0,1,0,0,0,0,1,0.454351,0.876131,0
2,3668-QPYBK,0,0.014085,0.354229,0.010310,1,1,0,0,1,...,0,0,0,1,0,0,1,0.542117,0.895177,0
3,7795-CFOCW,0,0.619718,0.239303,0.210241,0,1,0,0,0,...,0,1,0,0,0,0,0,0.385602,0.876287,0
4,9237-HQITU,0,0.014085,0.521891,0.015330,1,0,0,0,1,...,0,0,0,1,0,1,0,0.411883,0.894811,0
